In [6]:
# 🚗 Dynamic Pricing for Urban Parking Lots - Summer Analytics 2025
# Author: Tushar Sangwan
# Libraries: numpy, pandas only (Pathway & Bokeh later)

import numpy as np
import pandas as pd
from geopy.distance import geodesic

# ------------------------ ⚙️ Load Data ------------------------

df = pd.read_csv("dataset.csv")
print("Loaded dataset shape:", df.shape)

df = df.rename(columns={
    'SystemCodeNumber': 'lot_id',
    'Capacity': 'capacity',
    'Latitude': 'lat',
    'Longitude': 'lon',
    'Occupancy': 'occupancy',
    'VehicleType': 'vehicle_type',
    'TrafficConditionNearby': 'traffic_level',
    'QueueLength': 'queue_length',
    'IsSpecialDay': 'is_special_day'
})

df['vehicle_type'] = df['vehicle_type'].str.lower()
df['traffic_level'] = df['traffic_level'].map({'low': 0.3, 'medium': 0.6, 'high': 1.0})

# ------------------------ 🔢 Model 1 ------------------------

def model1_baseline_price(prev_price, occupancy, capacity, alpha=5):
    occ_ratio = occupancy / capacity
    return max(5, round(prev_price + alpha * occ_ratio, 2))

# ------------------------ 📈 Model 2 ------------------------

weights = {
    'occupancy': 1.0,
    'queue': 0.5,
    'traffic': 0.3,
    'special_day': 0.6,
    'vehicle_type': {'car': 0.2, 'bike': -0.1, 'truck': 0.4}
}

def compute_demand(row):
    occ_ratio = row['occupancy'] / row['capacity']
    vehicle_weight = weights['vehicle_type'].get(row['vehicle_type'], 0)
    return (
        weights['occupancy'] * occ_ratio +
        weights['queue'] * row['queue_length'] -
        weights['traffic'] * row['traffic_level'] +
        weights['special_day'] * row['is_special_day'] +
        vehicle_weight
    )

def demand_based_price(base_price, demand, scaler=1.2):
    norm_demand = demand - 0.5
    price = base_price * (1 + scaler * norm_demand)
    return round(min(max(price, base_price * 0.5), base_price * 2), 2)

# ------------------------ 🧠 Model 3 (Optimized) ------------------------

# Precompute nearby lots
unique_lots = df[['lot_id', 'lat', 'lon']].drop_duplicates()
nearby_map = {}

for i, lot in unique_lots.iterrows():
    current_coord = (lot['lat'], lot['lon'])
    lot_id = lot['lot_id']
    nearby_map[lot_id] = []

    for j, other in unique_lots.iterrows():
        if lot_id == other['lot_id']:
            continue
        dist = geodesic(current_coord, (other['lat'], other['lon'])).km
        if dist <= 0.5:
            nearby_map[lot_id].append(other['lot_id'])

def competitive_adjustment(curr_price, competitor_prices):
    if not competitor_prices:
        return curr_price
    avg_price = sum(competitor_prices) / len(competitor_prices)
    if avg_price < curr_price:
        return round(curr_price - 0.5, 2)
    elif avg_price > curr_price:
        return round(curr_price + 0.5, 2)
    return round(curr_price, 2)

def model3_final_price_fast(row, df_lots_grouped):
    demand = compute_demand(row)
    price = demand_based_price(10, demand)
    competitors = df_lots_grouped.get(row['lot_id'], pd.DataFrame())
    competitor_prices = competitors['model2_price'].tolist()
    return competitive_adjustment(price, competitor_prices)

# ------------------------ 🚀 Run All Models ------------------------

def run_all_models(df):
    df['baseline_price'] = df.apply(lambda r: model1_baseline_price(10, r['occupancy'], r['capacity']), axis=1)
    df['demand'] = df.apply(compute_demand, axis=1)
    df['model2_price'] = df['demand'].apply(lambda d: demand_based_price(10, d))

    # Precompute competitors for each lot
    df_lots_grouped = {}
    for lot_id in df['lot_id'].unique():
        nearby_lots = nearby_map.get(lot_id, [])
        competitors = df[df['lot_id'].isin(nearby_lots)]
        df_lots_grouped[lot_id] = competitors

    df['model3_price'] = df.apply(lambda r: model3_final_price_fast(r, df_lots_grouped), axis=1)
    return df

df_result = run_all_models(df)

# ------------------------ ✅ Output Preview ------------------------

print(df_result[['lot_id', 'occupancy', 'capacity', 'queue_length', 'model2_price', 'model3_price']].head())

# ------------------------ 💾 Export (Optional) ------------------------

# df_result.to_csv("dynamic_pricing_results.csv", index=False)


Loaded dataset shape: (18368, 12)
        lot_id  occupancy  capacity  queue_length  model2_price  model3_price
0  BHMBCCMKT01         61       577             1         12.59         12.59
1  BHMBCCMKT01         64       577             1         12.65         12.65
2  BHMBCCMKT01         80       577             2         18.98         18.98
3  BHMBCCMKT01        107       577             2         19.55         19.55
4  BHMBCCMKT01        150       577             2         16.84         16.84
